In [ ]:
node_shape: str = "ellipse"

In [ ]:
import graphviz
import re
from code_data_science import data_table as dt
import code_data_science.palette as palette

df = dt.read_csv("../samples/cobol_relationships.csv")

In [ ]:
graphviz.set_jupyter_format("svg")
graphviz.set_default_engine("sfdp")
dot = graphviz.Digraph("cobol-relationships", comment="COBOL relationships")

# clean dependent field remove all content up to first slash
df["dependent"] = df["dependent"].apply(lambda x: re.sub(r"^.*\/", "", x))


def style_node(isMissing, dependencyType):
    if isMissing == "true":
        return palette.__moderneColorMap["red"][200]
    if dependencyType == "COBOL":
        return palette.__moderneColorMap["blue"][400]
    if dependencyType == "COPYBOOK":
        return palette.__moderneColorMap["blue"][200]
    if dependencyType == "LINKEDIT":
        return palette.__moderneColorMap["yellow"][200]
    if dependencyType == "BINDPACKAGE":
        return palette.__moderneColorMap["green"][200]
    if dependencyType == "BINDPLAN":
        return palette.__moderneColorMap["green"][400]
    return "white"


def addAnnotation(value, docType):
    return f"<<b>{value}</b><br/><i>({docType})</i>>"


def map_relationship(row):
    dot.node(
        row["dependent"],
        shape=node_shape,
        label=addAnnotation(row["dependent"], row["dependentType"]),
        style="filled",
        fillcolor=style_node(row["dependencyMissing"], row["dependentType"]),
    )
    dot.node(
        row["dependency"],
        shape=node_shape,
        label=addAnnotation(row["dependency"], row["dependencyType"]),
        style="filled",
        fillcolor=style_node(row["dependencyMissing"], row["dependencyType"]),
    )
    dot.edge(row["dependent"], row["dependency"], row["action"])


df.apply(map_relationship, axis=1)

dot